In [2]:
import pandas as pd
from combat_unit import CombatUnit
from battle_analyzer import BattleAnalyzer
import enums
from IPython.display import display, HTML

units = pd.read_csv('data/unit_data.csv')
units['unit_id'] = units.index

an_units = units.loc[units['unit_type'] != 'Ground']

allied_units = an_units.loc[an_units['nationality'] != 'Japan']
japan_units = an_units.loc[an_units['nationality'] == 'Japan']

display(HTML(allied_units.to_html()))
display(HTML(japan_units.to_html()))

allied_unit_list = [CombatUnit(**kwargs) for kwargs in allied_units.to_dict(orient='records')]
japan_unit_list = [CombatUnit(**kwargs) for kwargs in japan_units.to_dict(orient='records')]

wasp = next((x for x in allied_unit_list if x.unit_id == 124), None)
york = next((x for x in allied_unit_list if x.unit_id == 114), None)

allied_forces = [wasp, york]
allied_forces[1].is_flipped = False

shokaku = next((x for x in japan_unit_list if x.unit_id == 95), None)
nagato = next((x for x in japan_unit_list if x.unit_id == 97), None)

japan_forces = [shokaku, nagato]
japan_forces[0].is_flipped = False

for unit in allied_forces:
    print(
        f'Unit: {unit.unit_name}, Front: {unit.attack_front}, Back: {unit.attack_back}, Defense: {unit.defense}, '
        f'CF: {unit.combat_factor()}, Loss Delta: {unit.loss_delta()}, Range: {unit.move_range}')

allied_forces_cf = sum(map(lambda x: x.combat_factor(), allied_forces))
print(f'Allied CF Total: {allied_forces_cf}')
print('')

for unit in japan_forces:
    print(
        f'Unit: {unit.unit_name}, Front: {unit.attack_front}, Back: {unit.attack_back}, Defense: {unit.defense}, '
        f'CF: {unit.combat_factor()}, Loss Delta: {unit.loss_delta()}, Range: {unit.move_range}')

japan_forces_cf = sum(map(lambda x: x.combat_factor(), japan_forces))
print(f'Japan CF Total: {japan_forces_cf}')

analyzer = BattleAnalyzer(intel_condition=enums.IntelCondition.INTERCEPT, reaction_player=enums.Player.ALLIES,
                          air_power_mod=enums.AirPowerModifier.Y1942, allied_ec_mod=0, japan_ec_mod=0)

results = analyzer.analyze_battle(allied_forces, japan_forces)

,nationality,unit_type,branch,attack_front,defense,attack_back,move_range,move_range_extended,extended_limit,unit_name,image_name_front,image_name_back,unit_id
45,US,Air,Army,8,10,4.0,2.0,4.0,True,FEAF,usfeafF.gif,usfeafB.gif,45
46,US,Air,Army,2,9,1.0,6.0,NaN,NaN,19 AF (LR),us19aflrbF.gif,us19aflrbB.gif,46
47,US,Air,Army,10,10,5.0,2.0,4.0,False,5 AF,us5afF.gif,us5afB.gif,47
48,US,Air,Army,10,10,5.0,2.0,4.0,False,7 AF,us7afF.gif,us7afB.gif,48
49,US,Air,Army,10,10,5.0,2.0,4.0,False,11 AF,us11afF.gif,us11afB.gif,49
50,US,Air,Army,10,10,5.0,2.0,4.0,False,13 AF,us13afF.gif,us13afB.gif,50
51,US,Air,Army,4,10,2.0,6.0,NaN,False,5 AF (LR),us5aflrbF.gif,us5aflrbB.gif,51
52,US,Air,Army,4,10,2.0,6.0,NaN,False,7 AF (LR),us7aflrbF.gif,us7aflrbB.gif,52
53,US,Air,Army,4,10,2.0,6.0,NaN,False,10 AF (LR),us10aflrbF.gif,us10aflrbB.gif,53
54,US,Air,Army,4,10,2.0,6.0,NaN,False,11 AF (LR),us11aflrbF.gif,us11aflrbB.gif,54


,nationality,unit_type,branch,attack_front,defense,attack_back,move_range,move_range_extended,extended_limit,unit_name,image_name_front,image_name_back,unit_id
7,Japan,Air,Army,20,10,10.0,2.0,4.0,False,1 AD,jpn1adF.gif,jpn1adB.gif,7
8,Japan,Air,Army,20,10,10.0,2.0,4.0,False,2 AD,jpn2adF.gif,jpn2adB.gif,8
9,Japan,Air,Army,20,10,10.0,2.0,4.0,False,3 AD,jpn3adF.gif,jpn3adB.gif,9
10,Japan,Air,Army,20,10,10.0,2.0,4.0,False,4 AD,jpn4adF.gif,jpn4adB.gif,10
11,Japan,Air,Army,22,10,11.0,2.0,4.0,False,5 AD,jpn5adF.gif,jpn5adB.gif,11
12,Japan,Air,Army,8,10,4.0,2.0,4.0,False,6 AD,jpn6adF.gif,jpn6adB.gif,12
13,Japan,Air,Army,8,10,4.0,2.0,4.0,False,7 AD,jpn7adF.gif,jpn7adB.gif,13
14,Japan,Air,Army,8,10,4.0,2.0,4.0,False,8 AD,jpn8adF.gif,jpn8adB.gif,14
15,Japan,Air,Army,8,10,4.0,2.0,4.0,False,9 AD,jpn9adF.gif,jpn9adB.gif,15
16,Japan,Air,Army,8,10,4.0,2.0,4.0,False,10 AD,jpn10adF.gif,jpn10adB.gif,16


Unit: Wasp (CV), Front: 12, Back: 8.0, Defense: 12, CF: 12, Loss Delta: 4.0, Range: 2.0
Unit: Duke of York (BB), Front: 20, Back: 10.0, Defense: 16, CF: 20, Loss Delta: 10.0, Range: nan
Allied CF Total: 32

Unit: Shokaku (CV), Front: 14, Back: 9.0, Defense: 12, CF: 14, Loss Delta: 5.0, Range: 3.0
Unit: Nagato (BB), Front: 20, Back: 10.0, Defense: 14, CF: 20, Loss Delta: 10.0, Range: nan
Japan CF Total: 34


In [3]:
results

,allied_die_roll,allied_result,allied_losses,allied_damage_applied,allied_remaining_cf,japan_die_roll,japan_result,japan_losses,japan_damage_applied,japan_remaining_cf,battle_winner
0,0,0.25,9,0,32,0,0.25,8,0,34,JAPAN
1,0,0.25,9,0,32,1,0.25,8,0,34,JAPAN
2,0,0.25,9,0,32,2,0.25,8,0,34,JAPAN
3,0,0.25,17,16,22,3,0.50,8,0,34,JAPAN
4,0,0.25,17,16,22,4,0.50,8,0,34,JAPAN
...,...,...,...,...,...,...,...,...,...,...,...
95,9,1.00,17,16,22,5,0.50,32,28,14,ALLIES
96,9,1.00,34,28,18,6,1.00,32,28,14,ALLIES
97,9,1.00,34,28,18,7,1.00,32,28,14,ALLIES
98,9,1.00,34,28,18,8,1.00,32,28,14,ALLIES


In [4]:
df_winner = results.groupby(['battle_winner']).agg(winner_count=('battle_winner', 'count'))

In [5]:
df_winner

,winner_count
battle_winner,
ALLIES,36
JAPAN,64


In [6]:
df_losses_allies = results.groupby(['allied_damage_applied'], as_index=False).agg(
    damage_count=('allied_damage_applied', 'count'))

df_losses_allies = df_losses_allies.rename(columns={'allied_damage_applied': 'damage_applied'})
df_losses_allies['player'] = enums.Player.ALLIES.name

df_losses_japan = results.groupby(['japan_damage_applied'], as_index=False).agg(
    damage_count=('japan_damage_applied', 'count'))

df_losses_japan = df_losses_japan.rename(columns={'japan_damage_applied': 'damage_applied'})
df_losses_japan['player'] = enums.Player.JAPAN.name

df_losses = pd.concat([df_losses_allies, df_losses_japan], ignore_index=True)

In [7]:
df_losses

,damage_applied,damage_count,player
0,0,30,ALLIES
1,16,30,ALLIES
2,28,30,ALLIES
3,32,10,ALLIES
4,0,30,JAPAN
5,14,30,JAPAN
6,26,30,JAPAN
7,28,10,JAPAN


In [8]:
df_pivot = pd.pivot_table(df_losses, index=['player'], columns=['damage_applied'], values=['damage_count'], aggfunc=sum, fill_value=0)

In [9]:
df_pivot

damage_count                    
damage_applied           0   14  16  26  28  32
player                                         
ALLIES                   30   0  30   0  30  10
JAPAN                    30  30   0  30  10   0

In [10]:
x_values = []
for x in df_pivot.columns:
    x_values.append(x[1])

pd.DataFrame(x_values)[0]

0     0
1    14
2    16
3    26
4    28
5    32
Name: 0, dtype: int64

In [11]:
pd.DataFrame(df_pivot.values[0])[0].apply(lambda z: z / 100)

0    0.3
1    0.0
2    0.3
3    0.0
4    0.3
5    0.1
Name: 0, dtype: float64

In [12]:
set_a = set([1, 2, 3, 4])
set_b = set([2, 3, 4])
set_a.symmetric_difference(set_b)

{1}

In [17]:
unit_ids = set(map(lambda x: x.unit_id, allied_forces))
unit_ids

{114, 124}

In [22]:
len(unit_ids)

0

In [21]:
unit_ids.pop()

124

In [25]:
child = {'props': {'id': 10}, 'type': 2, 'namespace': 3}
print(child['props']['id'])

10
